# UI

> A chat interface UI demo

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.utils import *
from fhbasecoat.common import *
from fhbasecoat.interactive import *
from fhbasecoat.chat import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from fastcore.utils import *
from itertools import product
from enum import Enum, auto
import nbformat
import itertools

In [ ]:
app = FastHTML(exts="ws", session_cookie="mysession")
rt = app.route

In [ ]:
# srv.stop()

In [ ]:
srv = JupyUvi(app)

In [ ]:
from pathlib import Path

In [ ]:
! ls ~/git/fhbasecoat/fhbasecoat

__init__.py  _modidx.py  common.py	 utils.py
__pycache__  chat.py	 interactive.py


In [ ]:
common_components = Path("~/git/fhbasecoat/fhbasecoat/common.py").expanduser().read_text()
interactive_components = Path("~/git/fhbasecoat/fhbasecoat/interactive.py").expanduser().read_text()

Note: whenever you answer questions from now you should refer to $`[common_components, interactive_components]`.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Got it! I can see you're working with FastHTML and have loaded some component code from `fhbasecoat`. I'll reference the `common_components` and `interactive_components` code when answering your questions.

What would you like to explore or work on with these components?

In [ ]:
def make_key():
    current = [None]
    def key_fn(cell):
        if cell.cell_type == "markdown" and cell.source.startswith("###"):
            current[0] = cell.source.strip("###").strip()
        return current[0]
    return key_fn

In [ ]:
def mk_nbdict(filename:str):
    nb = nbformat.read(filename, as_version=4)
    cells = itertools.islice(itertools.dropwhile(lambda c: c.source != "# Components", nb.cells), 1, None)
    return {header: list(group) for header, group in itertools.groupby(cells, key=make_key())}

In [ ]:
match = re.search(r"\d+_(\w+)\.ipynb", "01_common.ipynb").group(1)
match

'common'

In [ ]:
def format_nm(nm:str):
    return re.search(r"\d+_(\w+)\.ipynb", nm).group(1).title()

In [ ]:
documentation = {format_nm(nm): mk_nbdict(nm) for nm in ["01_common.ipynb", "02_interactive.ipynb", "03_chat.ipynb"]}

In [ ]:
documentation.keys()

dict_keys(['Common', 'Interactive', 'Chat'])

# Code

In [ ]:
ButtonGroup??


```python
def ButtonGroup(*args, cls="", **kwargs):
    return Div(*args, role="group", cls=f"button-group {cls}", **kwargs)
```

**File:** `~/git/fhbasecoat/fhbasecoat/common.py`

In [ ]:
def SearchBarButton(id):
    return DialogOpenButton(
        Div(
            Div(
                Icon("search", cls="size-4"),
                Span("Search Documentation", cls="text-sm"),
                cls="flex gap-2 items-center",
            ),
            Kbd("⌘ K", cls="text-2xs"),
            cls="flex items-center px-3 py-2 justify-between",
        ),
        cls="m-1 hover:cursor-pointer bg-background rounded-lg w-full",
        did=id,
    )

def SearchBarCommand(id):
    contents = []
    for category, nbdict in documentation.items():
        contents.append(
            CommandGroup(
                ItemHeader(category),
                *map(CommandItem, nbdict.keys()),
            )
        )
        contents.append(Separator())
    contents.pop()
    
    return CommandDialog(
        Command(
            CommandSearch(),
            CommandScrollable(
                *contents,
                style="height: 400px;",
            ),
            cls="w-4xl",
            style="top: 29%; max-width: 650px; max-height: 500px;",

        ),
        id=id,


    )


def SearchBar(id="search-bar"):
    return Div(
        SearchBarButton(id=id),
        SearchBarCommand(id=id),
        cls="flex w-full",
    )

In [ ]:
def DemoSidebar():
    return Sidebar(
        SearchBar(),
        Group(
            "Platform",
            SidebarCollapsable(IconTitle("Playground", icon="square-terminal"), ["History", "Starred", "Settings"]),
            SidebarCollapsable(IconTitle("Models", icon="bot"), ["Genesis", "Explorer", "Quantum"]),
            SidebarCollapsable(IconTitle("Documentation", icon="book-open"), ["Introduction", "Get Started", "Tutorials", "Changelog"]),
            SidebarCollapsable(IconTitle("Settings", icon="settings-2"), ["General", "Team", "Billing", "Limits"]),
        ),
        SidebarGroup("Projects", ["Design Engineering", "Sales & Marketing", "Travel"], icon_list=["frame", "chart-pie", "map"]),
        footer=Div(
                Dropdown(
                    DropdownItem("Upgrade to Pro"),
                    Separator(),
                    DropdownItem("Account"),
                    trigger_btn=DropdownTriggerButton(
                        Div(
                            Avatar(src="https://ui.shadcn.com/avatars/shadcn.jpg"), Span("Taya", cls="pl-2"),
                            cls="flex items-center w-full",
                        ),
                        Icon("chevrons-up-down"), did="sidebar-dropdown", 
                        cls=f"{ButtonT.ghost} justify-between py-6 px-3 m-1",
                        style="width: calc(100% - 0.5rem)"
                    ),
                    id="sidebar-dropdown",
                    side="right", align="end",
            ),
            cls="flex flex-col items-left w-full",
        ),
    )

pw(
    DemoSidebar()
)

I was just on a website where I could use ctrl + k hotkey to open search. Normally this opens chrome search but somehow this was overriden. How is that possible?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great question! Websites can override default browser keyboard shortcuts using JavaScript event listeners. They intercept the keypress before it reaches the browser's default handler.

The basic approach is to listen for keyboard events and call `preventDefault()` to stop the browser's default action. Here's a simple example:

```javascript
document.addEventListener('keydown', (e) => {
  if ((e.ctrlKey || e.metaKey) && e.key === 'k') {
    e.preventDefault();
    // Open your custom search here
  }
});
```

The key parts are:
- `e.ctrlKey` checks if Ctrl is pressed (or `e.metaKey` for Cmd on Mac)
- `e.key === 'k'` checks if the K key is pressed
- `e.preventDefault()` stops the browser's default behavior

Have you tried implementing something like this, or would you like to see how to integrate it with FastHTML components?

In [ ]:
def TopBar():
    return Div(
        Div(
            ToggleButton(),
            SeparatorVertical(),
            Breadcrumb(["Chats", "Demo Chat"]),
            cls="flex items-center gap-2"
        ),
        ThemeSwitcher(),
        cls="flex justify-between items-center w-full px-3 overflow-hidden py-1 border-b"
    )

pw(TopBar())

## Chat interface

In [ ]:
from lisette import *
chat = Chat(model="anthropic/claude-3-haiku-20240307")
chat

<lisette.core.Chat>

In [ ]:
# for r in chat("Testing2", stream=True):
#     delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
#     if delta and delta.content:
#         print(delta.content, end='', flush=True)
# final_response = r.choices[0].message.content

In [ ]:
@app.ws("/send_msg_ws")
async def send_msg_ws(text:str, send):
    await send(Div(ChatPrompt(text), id="chat-interface", hx_swap_oob="beforeend"))
    r = chat(text, stream=True)
    output = r.choices[0].message.content
    await send(Div(ChatAssistant(output), id="chat-interface", hx_swap_oob="beforeend"))

In [ ]:
#| export
def ChatMessage(content:str, msg_type="", cls="", rounding="", color=""): 
    return Div(
        Div(
            Div(
                P(msg_type, cls="ml-2"), cls=f"text-xs text-muted-background border-b border-border hover:cursor-pointer {rounding} rounded-b-none {color}"),
                Div(content, cls="px-4 py-3 whitespace-pre-wrap"),
        ),
        cls=f"border border-border text-sm {cls} {rounding}"
    )

def ChatPrompt(content:str):
    return ChatMessage(content, msg_type="Prompt", cls="bg-card w-[85%]", rounding="rounded-2xl rounded-tl-sm", color="bg-rose-700/70")

def ChatAssistant(content:str):
    return ChatMessage(content, msg_type="Assistant", cls="bg-background w-[90%] ml-auto mb-2", rounding="rounded-2xl", color="bg-rose-700/70")


In [ ]:
#| export
def ChatInterface(*contents, chat_input=None, cls="", id=None):
    return Div(
        Div(
            *contents,
            cls="bg-background border-l border-r flex flex-col flex-1 mx-1 p-3 gap-2",
            id=id,
        ),
        chat_input,
        cls=f"flex flex-col w-full lg:max-w-4xl mx-auto {cls}",
    )

In [ ]:
@rt
def demo_chat():
    return Div(
        DemoSidebar(),
        Div(
            TopBar(),
            ChatInterface(
                ChatPrompt("A test message sent by a user\n\nAnd new lines\nAre here."),
                ChatAssistant("A smaller response returned by a LLM. Great question. You are absolutely right!"),
                chat_input=ChatInput(
                    cls="m-1 mt-0", ta_cls="rounded-t-none", width="w-full",
                    hx_ext="ws", ws_connect="/send_msg_ws", ws_send=True,
                ),
                cls="flex-1",
                id="chat-interface",
            ),
            cls="h-full flex flex-col",
        ),
        cls="h-screen"
    )

## Things to add

- A llm backend integration that puts out prompt and assistant
- Backend websockets for streaming
- Token count in top panel using tiktoken (And token (i) hover showing cost)
- Click to edit prompt/assistant output
- LLM thinking mode button + display live thinking output
- Switch between prompt + Note
- Note Output
- Note output integration with 
- Python result using web-python interpreter
- Code output type
- Hotkeys for sidebar, submit, switch type, etc.

### Things I've learnt

When using flexboxes, a child has to have a parent with flex to be able to use it.
For dynamic styling you can do `w-full lg:max-w-4xl mx-auto` to have something dynamically resize from full to a larger size.

htmx things:
- hotkeys by doing `hx_trigger="keydown[ctrlKey && key=='Enter'] from:body`
- Reset form by doing `hx_on__after_request="this.reset()"`
- Button auto get `type="submit"` when inside a form.
- To allow trigger to work with clicks as well you can do `click from:button, keydown...`
- To disable during the request you can do `hx_disabled_elt="find button, find textarea"`
- To clear textarea you can do `hx_on__before_request="this.reset()"`